In [1]:
import pandas as pd
import ast
import pyGM as gm
import numpy as np

In [2]:
cards = pd.read_csv('data/cards.csv')

In [3]:
cards['CMC'] = cards['manaCost'].apply(lambda x: ''.join(''.join(str(x).split('{')).split('}')))
for col in cards:
    try:
        cards[col] = cards[col].apply(lambda x: ast.literal_eval(x))
    except:
        pass
for index, row in cards.iterrows():
    X = row['CMC']
    list(X)
    x_ = 0
    for x in X:
        try:
            x_+=int(x)
        except:
            x_+=1
    row['CMC']=x_

In [4]:
legal = []
for index, row in cards.iterrows():
    if 'modern' in row['legalities']:
        if row['legalities']['modern']=='Legal':
            legal.append(True)
        else:
            legal.append(False)
    else:
        legal.append(False)
cards = cards[legal]
cards.drop(columns=['legalities','Unnamed: 0','leadershipSkills','power','toughness','loyalty','uuid'],inplace=True)

In [5]:
cards.head()

,colors,manaCost,name,subtypes,supertypes,text,type,types,hasNoDeckLimit,CMC
8,[R],{2}{R},Abandon Reason,[],[],"[discard, discard, cast, madness]",Instant,[Instant],NaN,3
10,[],{3},Abandoned Sarcophagus,[],[],"[cast, cycling, cycling, exile]",Artifact,[Artifact],NaN,3
11,[B],{3}{B},Abattoir Ghoul,[Zombie],[],[creature],Creature — Zombie,[Creature],NaN,4
13,[W],{3}{W},Abbey Griffin,[Griffin],[],[vigilance],Creature — Griffin,[Creature],NaN,4
15,[R],{1}{R},Abbot of Keral Keep,"[Human, Monk]",[],"[cast, creature, exile, play]",Creature — Human Monk,[Creature],NaN,2


In [6]:
decks = dict()
decks['modern2018'] = pd.read_csv('data/deckListmodernMetaDecks2018.csv')
decks['modern2019'] = pd.read_csv('data/deckListmodernMetaDecks2019.csv')
decks['modern2020'] = pd.read_csv('data/deckListmodernMetaDecks2020.csv')


In [7]:
decklists = dict()
decklists['modern2020'] = decks['modern2020'][decks['modern2020']['1']=='Red Deck Wins']['3']
decklists['modern2019'] = decks['modern2019'][decks['modern2019']['1']=='Red Deck Wins']['3']
decklists['modern2018'] = decks['modern2018'][decks['modern2018']['1']=='Red Deck Wins']['3']


In [8]:
decklists['modern2020'] = decklists['modern2020'].to_dict()[0]
decklists['modern2020'] = ast.literal_eval(decklists['modern2020'])
decklists['modern2019'] = decklists['modern2019'].to_dict()[0]
decklists['modern2019'] = ast.literal_eval(decklists['modern2019'])
decklists['modern2018'] = decklists['modern2018'].to_dict()[1]
decklists['modern2018'] = ast.literal_eval(decklists['modern2018'])

In [9]:
simMain = dict()
simSide = dict()
for D in decklists:
    for i,deck in enumerate(decklists[D]):
        for ty in deck:
            if ty=='SIDEBOARD':
                sim=simSide
            else:
                sim=simMain
            for card in deck[ty]:
                card = card.split(' ')
                freq = card[0]
                name = ' '.join(card[1:])
                if name not in sim:
                    sim[name] = list()
                    if D=='modern2020':
                        i_ = i+60+80
                    if D=='modern2019':
                        i_ = i+80
                    if D=='modern2018':
                        i_ = i
                    sim[name].append([i_,int(freq)])
                else:
                    isIn = False
                    for i_,freq_ in sim[name]:
                        if i_==i:
                            isIn=True
                            freq_+=int(freq)
                    if isIn==False:
                        sim[name].append([i,int(freq)])

In [10]:
sim=simSide
columns=list(sim.keys())
d = pd.DataFrame(columns=columns)
for i in range(190+80+60):
    d.loc[i] = 0.0
for card in sim:
    for deck,freq in sim[card]:
        d.loc[deck][card]=freq
        
corr = d.corr()
for index, row in corr.iterrows():
    for i,r in enumerate(row):
        try:
            int(r)
        except:
            row[i]=-1
corrSide=corr
dSide = d

In [11]:
sim=simMain
columns=list(sim.keys())
d = pd.DataFrame(columns=columns)
for i in range(190+80+60):
    d.loc[i] = 0.0
for card in sim:
    for deck,freq in sim[card]:
        d.loc[deck][card]=freq
        
corr = d.corr()
for index, row in corr.iterrows():
    for i,r in enumerate(row):
        try:
            int(r)
        except:
            row[i]=-1
corrMain = corr
dMain = d

In [12]:
Decksize = 60
#For a given partial decklist, check if the addition of a new card
# satisfies the restrictions placed upon the deck by specifying a companion(s) for it
# and return True if the addition of a new card would not violate the deck restrictions
def meetsCRestrictions(Companions: list, Card: pd.Series(), Decklist: list) -> bool:
    for i in Companions:
        if i == "Lurrus of the Dream-Den" and ('Creature' in Card.types.tolist()[0] or 'Enchantment' in Card.types.tolist()[0] or 'Artifact' in Card.types.tolist()[0] or 'Planeswalker' in Card.types.tolist()[0]) and Card.CMC.tolist()[0] > 2:
            return False
        if i == "Keruga, the Macrosage" and 'Land' not in Card.types.tolist()[0] and Card.CMC.tolist()[0] < 3:
            return False
        if i == "Jegantha, the Wellspring" and Card.manaCost.tolist()[0].split('}') != len(set(Card.manaCost.tolist()[0].split('}'))):
            return False
        if i == "Obosh, the Preypiercer" and 'Land' not in Card.types.tolist()[0]  and Card.CMC.tolist()[0] %2 == 0:
            return False
        if i == "Gyruda, Doom of Depths" and 'Land' not in Card.types.tolist()[0]  and Card.CMC.tolist()[0] % 2 == 1:
            return False
        if i == "Umori, the Collector":
            CardType = None
            for ty in Decklist:
                for x in Decklist[ty]:
                    name = ' '.join(x.split()[1:])
                    x = cards[cards['name']==name]
                    if CardType == None:
                        CardType = x.types.tolist()[0]
                    elif 'Land' in CardType:
                        CardType = x.types.tolist()[0]
                    elif len(x.types.tolist()[0])>1 and 'Land' not in x.types.tolist()[0]:
                        return False
                    else:
                        if x.types.tolist()[0] != ["Land"] and x.types.tolist()[0] not in CardType:
                            return False
        if i == "Lutri, the Spellchaser":
            for ty in Decklist:
                for x in Decklist[ty]:
                    freq = int(x.split()[0])
                    name  = ' '.join(x.split()[1:])
                    if (freq >1 and ty!='LANDS') or name==Card.name.tolist()[0]:
                        return False

        if i == "Kaheera, the Orphaguard" and 'Creature' in Card.types.tolist()[0]  and ("Cat" not in Card.subtypes.tolist()[0]  or "Elemental" not in Card.subtypes.tolist()[0] or "Nightmare" not in Card.subtypes.tolist()[0] or "Dinosaur" not in Card.subtypes.tolist()[0] or "Beast" not in Card.subtypes.tolist()[0]):
            return False
        if i == "Yorion, Sky Nomad":
            Decksize = 80
    return True

C:\Users\joey\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


In [13]:
sampleDecklist = {}
sampleDecklist["LANDS"] = []
sampleDecklist["CREATURES"] = ["4 Monastery Swiftspear", "2 Magus of the Moon", "4 Goblin Guide"]
sampleDecklist["INSTANTS and SORC."] = ["4 Lightning Bolt", "4 Boros Charm"]
# sampleDecklist["SIDEBOARD"] = ["1 Lurrus of the Dream-Den"]
name = 'Lava Axe'
comp = "Kaheera, the Orphaguard"
meetsCRestrictions([comp],cards[cards['name']==name],Decklist=sampleDecklist)


True

In [14]:
from Copy import copy
def GetRecommended(Data: pd.DataFrame(), IncDeck: dict) -> dict:
    recommendationDeck = pd.Series()
    mainDeck = copy.deepcopy(IncDeck)
    currSize = 0
    if 'SIDEBOARD' in IncDeck.keys():
        del mainDeck['SIDEBOARD']
        sideDeck = IncDeck['SIDEBOARD']
        for x in sideDeck:
            x = ' '.join(x.split()[1:])
            potentialCard = cards[cards['name']==x]
    for i in mainDeck.values():
        nameVal = i.split()
        CurrSize += nameVal[0]
    print("Current Deck Size: " + str(CurrSize))
    while CurrSize < Decksize:
        

SyntaxError: unexpected EOF while parsing (<ipython-input-14-4f6b8f4e10c7>, line 17)